# Selenium 기초

In [2]:
# 목적 : 인터넷 상의 데이터 수집
# librar: selenium, webdriver-manager

In [1]:
!pip install selenium

In [2]:
# chrome driver
!pip install webdriver-manager 

In [3]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [4]:
ChromeDriverManager().install()
#내 컴퓨터에 chrome driver를 자동으로 다운받아 줍니다.

'C:\\Users\\PC\\.wdm\\drivers\\chromedriver\\win64\\124.0.6367.91\\chromedriver-win32/chromedriver.exe'

In [5]:
browser = webdriver.Chrome()

In [6]:
# 위에서 열어놓은 크롬이 닫힌상태에서 이 코드를 실행하면 에러뜬다.
url = 'https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15098789'
browser.get(url)  

In [7]:
browser.title

'보건복지부_보건·복지현황_주요 사망 원인별 사망자 수 | 공공데이터포털'

In [8]:
# 크롤링 하려면 요소(element)를 잘 찾아야 합니다
from selenium.webdriver.common.by import By

In [16]:
item_name = {}
for i in range(1,28):
    str1 = f'//*[@id="open-api-detail-result"]/div[3]/table/tbody/tr[{i}]/td[1]'
    str2 = f'//*[@id="open-api-detail-result"]/div[3]/table/tbody/tr[{i}]/td[2]'
    key = browser.find_element(By.XPATH, str2).text
    value = browser.find_element(By.XPATH, str1).text
    if '_' in value:
        item_name[key] = value.split('_')[1]
    else:
        item_name[key] = value
item_name

{'resultCode': '결과코드',
 'resultMsg': '결과메시지',
 'numOfRows': '한 페이지 결과 수',
 'pageNo': '페이지 번호',
 'totalCount': '전체 결과 수',
 'mrdhErds': '귀 및 유돌의 질환',
 'mrdhPrgps': '임신, 출산 및 산후기',
 'mrdhSytcls': '달리 분류되지 않은 증상, 징후',
 'mrdhDsrp': '호흡기계통의 질환',
 'mrdhDskts': '피부 및 피하조직의 질환',
 'mrdhCmca': '선천기형, 변형 및 염색체 이상',
 'year': '년도',
 'mrdhCropr': '출생전후기에 기원한 특정병태',
 'mrdhDscry': '순환기계통의 질환',
 'mrdhOtsd': '질병이환 및 사망의 외인',
 'mrdhEyds': '눈 및 눈부속기의 질환',
 'mrdhDsdst': '소화기계통의 질환',
 'mrdhDsbh': '혈액 및 조혈기관질환과 면역메커니즘을 침범하는 특정장애',
 'mrdhMtbh': '정신 및 행동장애',
 'mrdhDsgtr': '비뇨생식기계통의 질환',
 'dvs': '구분',
 'mnCsdthCrpds': '특정감염성 및 기생충성 질환',
 'mrdhDsms': '근육골격계통 및 결합 조직의 질환',
 'mrdhDsnrs': '신경계통의 질환',
 'mrdhEdmds': '내분비, 영양 및 대사질환',
 'mrdhTtl': '계',
 'mrdhNplsm': '신생물'}

In [30]:
import xml.etree.ElementTree as ET
import pandas as pd
import requests

url = 'http://apis.data.go.kr/1352000/ODMS_STAT_05/callStat05Api'

df = pd.DataFrame()

for year in range(2005,2020):
    params ={'serviceKey' : 'i+ggaeuJtzvMkgkOEQEP5AJ9X2EbQu3TWoi7HWeP6xyJt/jpS60YgqpL9MDmZLyR4Gl84TSPnDEs9JvrJUCAoQ==', 'pageNo' : '1', 'numOfRows' : '10', 'apiType' : 'XML', 'year' : f'{year}', 'dvs' : '남자' }
    response = requests.get(url, params=params)
    print(response.content)
    
    # XML 파싱
    root = ET.fromstring(response.content)
    items = root.findall('.//item')
    
    # DataFrame으로 변환
    data = []
    for item in items:
        row = {}
        for child in item:   
            row[item_name[child.tag]] = child.text
            print(item_name[child.tag], child.tag, child.text)
        data.append(row)
    df = pd.concat([df, pd.DataFrame(data)], axis=0)

b'<?xml version="1.0" encoding="UTF-8"?>\r\n<response>\r\n <header>\r\n  <resultCode>00</resultCode>\r\n  <resultMsg>NORMAL SERVICE</resultMsg>\r\n </header>\r\n <body>\r\n  <items>\r\n   <item>\r\n    <year>2005</year>\r\n    <dvs>\xeb\x82\xa8\xec\x9e\x90</dvs>\r\n    <mrdhTtl>134382</mrdhTtl>\r\n    <mnCsdthCrpds>3409</mnCsdthCrpds>\r\n    <mrdhNplsm>41481</mrdhNplsm>\r\n    <mrdhDsbh>207</mrdhDsbh>\r\n    <mrdhEdmds>6271</mrdhEdmds>\r\n    <mrdhMtbh>1940</mrdhMtbh>\r\n    <mrdhDsnrs>1987</mrdhDsnrs>\r\n    <mrdhEyds>null</mrdhEyds>\r\n    <mrdhErds>2</mrdhErds>\r\n    <mrdhDscry>27171</mrdhDscry>\r\n    <mrdhDsrp>8387</mrdhDsrp>\r\n    <mrdhDsdst>8118</mrdhDsdst>\r\n    <mrdhDskts>152</mrdhDskts>\r\n    <mrdhDsms>414</mrdhDsms>\r\n    <mrdhDsgtr>1526</mrdhDsgtr>\r\n    <mrdhPrgps>null</mrdhPrgps>\r\n    <mrdhCropr>476</mrdhCropr>\r\n    <mrdhSytcls>304</mrdhSytcls>\r\n    <mrdhCmca>11551</mrdhCmca>\r\n    <mrdhOtsd>20986</mrdhOtsd>\r\n   </item>\r\n  </items>\r\n  <numOfRows>10</num

In [29]:
df

,년도,구분,계,특정감염성 및 기생충성 질환,신생물,혈액 및 조혈기관질환과 면역메커니즘을 침범하는 특정장애,"내분비, 영양 및 대사질환",정신 및 행동장애,신경계통의 질환,눈 및 눈부속기의 질환,...,호흡기계통의 질환,소화기계통의 질환,피부 및 피하조직의 질환,근육골격계통 및 결합 조직의 질환,비뇨생식기계통의 질환,"임신, 출산 및 산후기",출생전후기에 기원한 특정병태,"달리 분류되지 않은 증상, 징후","선천기형, 변형 및 염색체 이상",질병이환 및 사망의 외인
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df.to_csv('2005to2019_연도별_질병별_사망자수.csv', encoding='utf-8-sig')